In [2]:
!pip install cog

  Using cached cog-0.9.13-py3-none-any.whl.metadata (38 kB)
  Using cached fastapi-0.98.0-py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 4.0 MB/s eta 0:00:00a 0:00:01
  Using cached structlog-24.4.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached uvicorn-0.30.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached starlette-0.27.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached httptools-0.6.1-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.6 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached uvloop-0.19.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.9 kB)
  Using cached watchfiles-0.22.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.9 kB)
  Using cached anyio-4.4.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached sniffio-1.3.1-py3

In [18]:
# Imports 
import json
import csv
import time
import os
import argparse
import random
import requests
import itertools
from typing import Dict, Any, List, Set, Tuple, Optional
from whisperx_server import WhisperServer, Output
import logging
import sys
import json
import os
import random
import requests
import pandas as pd
from IPython.display import Audio

In [11]:
# Constants
TEST_CASES_FILE = "./test-openai-whisper-testcases.json"
TEMP_AUDIO_DIR = "tests/temp_audio"
RESULTS_DIR = "./crossproduct-experiment-results"
RESULTS_CSV = f"{RESULTS_DIR}/whisperx_performance_test_results.csv"

In [12]:
# Load test cases
with open(TEST_CASES_FILE, "r") as f:
    test_cases = json.load(f)

# Create temporary audio directory if it doesn't exist
os.makedirs(TEMP_AUDIO_DIR, exist_ok=True)

In [15]:
# Download audio files
for test_case in test_cases:
    audio_url = test_case["audio"]
    file_name = os.path.basename(audio_url)
    file_path = os.path.join(TEMP_AUDIO_DIR, file_name)
    
    print(f"\n🔍 Processing test case: {test_case}")
    print(f"🎵 Audio URL: {audio_url}")
    print(f"📄 File name: {file_name}")
    print(f"📂 File path: {file_path}")

    if not os.path.exists(file_path):
        print(f"❌ File does not exist. Downloading from {audio_url}...")
        response = requests.get(audio_url)
        
        if response.status_code == 200:
            with open(file_path, "wb") as f:
                f.write(response.content)
            print(f"✅ Downloaded and saved to {file_path}")
        else:
            print(f"🚨 Failed to download file. Status code: {response.status_code}")
    else:
        print(f"✅ File already exists at {file_path}. Skipping download.")


🔍 Processing test case: {'audio': 'https://replicate.delivery/mgxm/d9fa255c-4c47-4fec-99ce-f190539f10c4/olle.mp3', 'translate': True, 'expected_language': 'swedish', 'name': 'olle', 'expected_transcription': ' Man har sett alla folkets hus, alla Gotemplagårdar som de får spela i, alla CUF-gårdar. Ja, naturligtvis alla restauranger runt omkring. Man säger 70- och 80-tal så var det ju väldigt mycket på torsdagskvällen när man spelade på Krogan. För då var det ju resande ute och dansade och det är ju slut med det idag.', 'expected_translation': " You've seen all the people's houses, all the temple gardens that they play in, all the CF-gards. And of course all the restaurants around. In the 70's and 80's there was a lot of Thursday nights when you played at the pubs, because then it was traveling and dancing. It's over with that today."}
🎵 Audio URL: https://replicate.delivery/mgxm/d9fa255c-4c47-4fec-99ce-f190539f10c4/olle.mp3
📄 File name: olle.mp3
📂 File path: tests/temp_audio/olle.mp3
✅

In [31]:
!tree tests/temp_audio/

tests/temp_audio/
├── 4th-dimension-explained-by-a-high-school-student.mp3
├── OSR_cn_000_0072_8k.wav
├── OSR_uk_000_0050_8k.wav
├── Untitled.wav
├── bruce.mp3
├── nyn_konge02_1.mp3
└── olle.mp3

0 directories, 7 files


In [19]:
# Showcase one of the audio files
random_audio = random.choice(test_cases)
audio_file = os.path.join(TEMP_AUDIO_DIR, os.path.basename(random_audio["audio"]))
Audio(audio_file)

In [27]:
# Define the columns for the CSV
CSV_COLUMNS = [
    "audio",
    "model",
    "transcription",
    "translate",
    "language",
    "temperature",
    "patience",
    "suppress_tokens",
    "initial_prompt",
    "condition_on_previous_text",
    "temperature_increment_on_fallback",
    "compression_ratio_threshold",
    "logprob_threshold",
    "no_speech_threshold",
    "batch_size",
    "processing_time",
    "detected_language",
    "status",
    "expected_language",
    "expected_transcription",
    "expected_translation",
]

In [28]:
# Create an empty DataFrame
columns = ["file_name", "length_of_input_audio", "url"] + CSV_COLUMNS + ["processing_time", "succeeded", "logs"]
df = pd.DataFrame(columns=columns)
df

,file_name,length_of_input_audio,url,audio,model,transcription,translate,language,temperature,patience,...,batch_size,processing_time,detected_language,status,expected_language,expected_transcription,expected_translation,processing_time,succeeded,logs


In [ ]:
server = ExtendedWhisperServer()


In [ ]:




# Define the columns for the CSV
CSV_COLUMNS = [
    "audio",
    "model",
    "transcription",
    "translate",
    "language",
    "temperature",
    "patience",
    "suppress_tokens",
    "initial_prompt",
    "condition_on_previous_text",
    "temperature_increment_on_fallback",
    "compression_ratio_threshold",
    "logprob_threshold",
    "no_speech_threshold",
    "batch_size",
    "processing_time",
    "detected_language",
    "status",
    "expected_language",
    "expected_transcription",
    "expected_translation",
]

param_ranges = {
    "language": [
        "auto",
        "sv",
        "zh",
        "fr",
        "no",
        "en",
        "ja",
        "ko",
        "de",
        "es",
        "it",
        "nl",
        "pl",
        "pt",
        "ru",
        "ar",
        "hi",
    ],
    "translate": [True, False],
    "transcription": ["plain text", "srt", "vtt"],
    "temperature": [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0],
    "patience": [None, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0],
    "suppress_tokens": ["-1", "-1,0", "", "0,1,2,3,4,5,6,7,8,9", "264"],
    "initial_prompt": [
        None,
        "This is a test prompt",
        "Please transcribe the following audio accurately:",
        "Ignore any background noise and focus on the main speaker.",
    ],
    "condition_on_previous_text": [True, False],
    "temperature_increment_on_fallback": [0.1, 0.2, 0.3, 0.4, 0.5],
    "compression_ratio_threshold": [1.5, 2.0, 2.5, 3.0, 3.5, 4.0],
    "logprob_threshold": [-2.0, -1.5, -1.0, -0.5, 0.0],
    "no_speech_threshold": [0.2, 0.4, 0.6, 0.8, 1.0],
    "batch_size": [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024],
    "model": ["large-v3"],
}


class ExtendedWhisperServer(WhisperServer):
    def make_prediction(self, audio_url, **kwargs) -> Dict[str, Any]:
        payload = {"input": {"audio": audio_url, "model": "large-v3", **kwargs}}
        response = requests.post(self.SERVER_URL, json=payload)  # IO Blocking
        json_response = response.json()

        print(f"\nTest {kwargs.get('name', '')}")
        print(f"Detected Language: {json_response['output']['detected_language']}")
        print(f"Transcription: {json_response['output']['transcription']}")
        print(f"Translation: {json_response['output'].get('translation', 'N/A')}")
        print(f"Processing Time: {json_response['output']['processing_time']} seconds")

        return json_response


def load_test_cases() -> List[Dict[str, Any]]:
    """Load all test cases from the JSON file."""
    print("Loading test cases from JSON file...")
    with open(TEST_CASES_FILE, "r") as f:
        all_cases = json.load(f)
    print(f"Loaded {len(all_cases)} test cases.")
    return all_cases


def load_completed_tests() -> Set[Tuple]:
    """Load the parameter combinations of completed tests from the CSV file."""
    completed_tests = set()
    if os.path.exists(RESULTS_CSV):
        with open(RESULTS_CSV, "r") as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                test_params = tuple(row[param] for param in param_ranges.keys())
                completed_tests.add(test_params)
    return completed_tests


def run_test_case(
    server: ExtendedWhisperServer,
    test_case: Dict[str, Any],
    params: Dict[str, Any],
) -> Optional[Dict[str, Any]]:
    """Run a single test case and return the results, or None if there's an error."""
    print(f"\nRunning test case: {test_case['name']}")
    logging.info(f"\nRunning test case: {test_case['name']}")
    print("Parameters:")
    for key, value in params.items():
        print(f"  {key}: {value}")
    print(f"  audio: {test_case['audio']}")

    try:
        print("Overriding language and translate parameters if specified...")
        if "expected_language" in test_case:
            params["language"] = test_case["expected_language"]
        if "translate" in test_case:
            params["translate"] = test_case["translate"]

        print("Making prediction...")
        response = server.make_prediction(
            audio_url=test_case["audio"], name=test_case["name"], **params
        )
        print("Prediction made successfully")

        print("Preparing result dictionary...")
        result_dict = {
            "audio": test_case["audio"],
            "model": response["input"]["model"],
            **{
                k: response["input"].get(k, None)
                for k in CSV_COLUMNS
                if k in response["input"]
            },
            "processing_time": response["output"]["processing_time"],
            "detected_language": response["output"]["detected_language"],
            "status": response["status"],
            "expected_language": test_case.get("expected_language"),
            "expected_transcription": test_case.get("expected_transcription"),
            "expected_translation": test_case.get("expected_translation"),
        }
        print(
            f"Test case {test_case['name']} completed with status: {result_dict['status']}"
        )
        logging.info(
            f"Test case {test_case['name']} completed with status: {result_dict['status']}"
        )
        return result_dict
    except requests.RequestException as e:
        print(f"Network error in test case {test_case['name']}: {str(e)}")
        logging.error(f"Network error in test case {test_case['name']}: {str(e)}")
    except KeyError as e:
        print(f"Missing key in response for test case {test_case['name']}: {str(e)}")
        logging.error(
            f"Missing key in response for test case {test_case['name']}: {str(e)}"
        )
    except Exception as e:
        print(f"Unexpected error in test case {test_case['name']}: {str(e)}")
        logging.error(f"Unexpected error in test case {test_case['name']}: {str(e)}")
    return None


def write_result_to_csv(result: Dict[str, Any]):
    """Append a single result to the CSV file."""
    os.makedirs(RESULTS_DIR, exist_ok=True)

    file_exists = os.path.isfile(RESULTS_CSV)

    with open(RESULTS_CSV, "a", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=CSV_COLUMNS)
        if not file_exists:
            writer.writeheader()
        writer.writerow(result)
    print(f"Result written to {RESULTS_CSV}")


# Set up logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)


def parameter_combination_generator(param_ranges):
    keys = list(param_ranges.keys())
    values = list(param_ranges.values())
    for combination in itertools.product(*values):
        yield dict(zip(keys, combination))


def main(continue_from_last: bool, warm_up: bool = True, num_tests: int = 100) -> None:
    try:
        print("Starting main function")
        logging.info("Setting up WhisperX server...")
        server = ExtendedWhisperServer()
        print("WhisperX server set up")
        logging.info("WhisperX server set up successfully.")

        if warm_up:
            logging.info("Starting warm-up...")
            server.warm_up()
            logging.info("Warm-up completed.")
        else:
            print("Skipping warm-up")

        print("Loading test cases...")
        test_cases = load_test_cases()
        print(f"Loaded {len(test_cases)} test cases")

        print("Preparing parameter combinations generator...")
        combinations_generator = parameter_combination_generator(param_ranges)

        completed_tests = set()
        if continue_from_last:
            print("Loading completed tests...")
            completed_tests = load_completed_tests()
            print(f"Loaded {len(completed_tests)} completed tests")
            logging.info(f"Loaded {len(completed_tests)} completed tests.")

        print(f"Ensuring results directory exists: {RESULTS_DIR}")
        os.makedirs(RESULTS_DIR, exist_ok=True)
        logging.info(f"Results directory ensured: {RESULTS_DIR}")

        tests_run = 0
        print(f"Starting test loop, will run up to {num_tests} tests")
        for i, params in enumerate(combinations_generator):
            if tests_run >= num_tests:
                print(f"Reached {num_tests} tests, breaking loop")
                break

            if continue_from_last and tuple(params.values()) in completed_tests:
                continue

            print(f"\nPreparing test {tests_run+1}/{num_tests} (combination {i+1})")
            test_case = random.choice(test_cases)
            print(f"Selected test case: {test_case['name']}")
            logging.info(f"\nStarting test {tests_run+1}/{num_tests}")

            print("Running test case...")
            result = run_test_case(server, test_case, params)
            if result is not None:
                print(f"Test result status: {result['status']}")
                if result["status"] == "succeeded":
                    print("Writing result to CSV...")
                    write_result_to_csv(result)
                    tests_run += 1
                    print(f"Tests completed: {tests_run}")
                else:
                    print(f"Test did not succeed, status: {result['status']}")
            else:
                print("Test result is None")
                logging.warning(
                    f"Skipped writing result for test case {test_case['name']} due to error or unsuccessful status."
                )

            print("Sleeping for 0.1 seconds...")
            time.sleep(
                0.1
            )  # Small delay between tests to avoid overwhelming the server

        print(f"Test loop completed. Total tests run: {tests_run}")
        logging.info(f"Completed {tests_run} tests successfully.")
    except Exception as e:
        print(f"An error occurred in main: {str(e)}")
        logging.exception(f"An error occurred in main: {str(e)}")
        raise

    print("Main function completed successfully")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Run WhisperX performance tests")
    parser.add_argument(
        "--continue",
        dest="continue_from_last",
        action="store_true",
        help="Continue from last completed test",
    )
    parser.add_argument(
        "--no-warm-up",
        dest="warm_up",
        action="store_false",
        help="Skip warming up the server",
    )
    parser.add_argument(
        "--num-tests",
        type=int,
        default=100,
        help="Number of tests to run before stopping",
    )
    args = parser.parse_args()

    try:
        main(args.continue_from_last, args.warm_up, args.num_tests)
    except KeyboardInterrupt:
        logging.info("\nTest execution stopped by user.")
    except Exception as e:
        logging.exception(f"\nAn unexpected error occurred: {e}")
    finally:
        logging.info("Exiting the program.")
